<a href="https://colab.research.google.com/github/felipepizzinato/challenge-ia-2tdsph/blob/main/teste_challenge_2tds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

!pip install easyocr pandas opencv-python-headless
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install scikit-learn joblib


Cloning into 'yolov5'...
remote: Enumerating objects: 17485, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 17485 (delta 78), reused 30 (delta 30), pack-reused 17378 (from 3)
Receiving objects: 100% (17485/17485), 16.35 MiB | 17.51 MiB/s, done.
Resolving deltas: 100% (11989/11989), done.
/content/yolov5/yolov5
Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import torch
import easyocr
import os
import pandas as pd
from PIL import Image
import re

# Carrega modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.conf = 0.4
reader = easyocr.Reader(['en'])

# Caminho das imagens
caminho_pasta = '/content/imagens/'
arquivos = os.listdir(caminho_pasta)
imagens = [f for f in arquivos if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Controle de IDs
placas_detectadas = []
placa_to_id = {}
id_counter = 0

# Palavras comuns que devem ser ignoradas
palavras_proibidas = {"BRASIL", "BR", "MERCOSUL", "ASIL", "BEASIL"}

def combinar_palavras_em_placa(placas):
    """ Tenta formar uma placa provável a partir de combinações """
    # Remove palavras curtas e proibidas
    palavras_validas = [p for p in placas if p not in palavras_proibidas and len(p) >= 3]
    if not palavras_validas:
        return None
    # Junta todas as palavras e tenta formar algo parecido com placa
    combinado = ''.join(palavras_validas)
    if len(combinado) >= 5:
        return combinado
    return None

def extrair_placas(textos):
    placas = []
    for texto in textos:
        texto_limpo = re.sub(r'[^A-Z0-9]', '', texto.upper())
        if texto_limpo and texto_limpo not in palavras_proibidas:
            placas.append(texto_limpo)
    return placas if placas else ["Não identificado"]

def escolher_placa_valida(placas):
    for p in placas:
        if re.fullmatch(r'[A-Z]{3}[0-9][A-Z0-9][0-9]{2}', p):  # Ex: ABC1D23
            return p
    for p in placas:
        if len(p) >= 5:
            return p
    return None

for nome_arquivo in imagens:
    caminho_completo = os.path.join(caminho_pasta, nome_arquivo)
    image = Image.open(caminho_completo)

    results = model(caminho_completo)
    detections = results.pandas().xyxy[0]
    moto_detections = detections[detections['name'] == 'motorcycle']

    for i, row in moto_detections.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        cropped = image.crop((x1, y1, x2, y2))
        cropped_path = f"recortes/{nome_arquivo}_moto_{i}.jpg"
        os.makedirs("recortes", exist_ok=True)
        cropped.save(cropped_path)

        ocr_result = reader.readtext(cropped_path)
        textos = [text[1] for text in ocr_result]
        placas = extrair_placas(textos)
        placa_montada = combinar_palavras_em_placa(placas)

        if not placa_montada:
          bbox_hash = f"{nome_arquivo}_{x1}_{y1}_{x2}_{y2}"
          if bbox_hash not in placa_to_id:
              placa_to_id[bbox_hash] = id_counter
              id_counter += 1
          moto_id = placa_to_id[bbox_hash]
        else:
            if placa_montada in placa_to_id:
                moto_id = placa_to_id[placa_montada]
            else:
                moto_id = id_counter
                id_counter += 1
                placa_to_id[placa_montada] = moto_id

        placas_detectadas.append({
            'arquivo': nome_arquivo,
            'moto_id': moto_id,
            'placas': placas,
            'placa_combinada': placa_montada if placa_montada else "Não identificada",
            'bbox_x1': x1,
            'bbox_y1': y1,
            'bbox_x2': x2,
            'bbox_y2': y2
        })

df = pd.DataFrame(placas_detectadas)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-20 Python-3.11.12 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 64.7MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralyt

In [ ]:
import joblib
import numpy as np


def contar_letras_digitos(placa):
    if placa == "Não identificada" or pd.isna(placa):
        return (0, 0)
    letras = sum(c.isalpha() for c in placa)
    digitos = sum(c.isdigit() for c in placa)
    return letras, digitos

# Tamanho da placa
df['placa_len'] = df['placa_combinada'].fillna('').apply(len)

# Letras e dígitos
df[['n_letras', 'n_digitos']] = df['placa_combinada'].fillna('').apply(lambda p: pd.Series(contar_letras_digitos(p)))

# Tamanho do bounding box
df['bbox_area'] = (df['bbox_x2'] - df['bbox_x1']) * (df['bbox_y2'] - df['bbox_y1'])

# Carrega o modelo treinado
scaler = joblib.load('/content/scaler_kmeans.pkl')
modelo_kmeans = joblib.load('/content/modelo_kmeans_placas.pkl')

# Prepara os dados de entrada
features  = df[['placa_len', 'n_letras', 'n_digitos', 'bbox_area']]

# Aplica o modelo
features_scaled = scaler.transform(features)
df['grupo_placa'] = modelo_kmeans.predict(features_scaled)

df['grupo_placa'].value_counts()

df.to_csv("/content/resultado_placas.csv", index=False)

print("Finalizado com sucesso.")


Finalizado com sucesso.


In [ ]:
import pandas as pd

GRID_ROWS = 4
GRID_COLS = 4
IMG_WIDTH = 1600
IMG_HEIGHT = 720

df = pd.read_csv('/content/resultado_placas.csv')

df['center_x'] = (df['bbox_x1'] + df['bbox_x2']) / 2
df['center_y'] = (df['bbox_y1'] + df['bbox_y2']) / 2

df['grid_col'] = (df['center_x'] / IMG_WIDTH * GRID_COLS).astype(int)
df['grid_row'] = (df['center_y'] / IMG_HEIGHT * GRID_ROWS).astype(int)

df['grid_col'] = df['grid_col'].clip(0, GRID_COLS - 1)
df['grid_row'] = df['grid_row'].clip(0, GRID_ROWS - 1)

for arquivo, group in df.groupby('arquivo'):
    print(f"\nImagem: {arquivo}")
    for _, row in group.iterrows():
        print(f"Moto ID {row['moto_id']}: Grid ({row['grid_row']}, {row['grid_col']})")



Imagem: WhatsApp Image 2025-05-15 at 22.57.05.jpeg
Moto ID 0: Grid (1, 1)
Moto ID 1: Grid (1, 2)
Moto ID 2: Grid (2, 0)
Moto ID 3: Grid (0, 3)

Imagem: WhatsApp Image 2025-05-15 at 22.57.06.jpeg
Moto ID 2: Grid (3, 0)
Moto ID 2: Grid (3, 0)
Moto ID 2: Grid (3, 0)
Moto ID 2: Grid (3, 0)
Moto ID 2: Grid (3, 0)

Imagem: WhatsApp Image 2025-05-15 at 22.57.07.jpeg
Moto ID 2: Grid (2, 2)
Moto ID 2: Grid (2, 2)
Moto ID 2: Grid (2, 1)
Moto ID 2: Grid (1, 2)
Moto ID 9: Grid (2, 0)
Moto ID 2: Grid (1, 2)

Imagem: WhatsApp Image 2025-05-15 at 22.57.08.jpeg
Moto ID 4: Grid (1, 2)
Moto ID 5: Grid (1, 1)
Moto ID 6: Grid (2, 2)
Moto ID 7: Grid (1, 1)

Imagem: imagem.jpeg
Moto ID 2: Grid (3, 1)
Moto ID 2: Grid (3, 0)
Moto ID 8: Grid (3, 0)
Moto ID 2: Grid (3, 0)
Moto ID 2: Grid (3, 0)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os

GRID_ROWS = 4
GRID_COLS = 4
IMG_WIDTH = 1600
IMG_HEIGHT = 720

output_dir = "/content/data/grid_images"
os.makedirs(output_dir, exist_ok=True)

csv_path = "/content/resultado_placas.csv"
df = pd.read_csv(csv_path)

df['center_x'] = (df['bbox_x1'] + df['bbox_x2']) / 2
df['center_y'] = (df['bbox_y1'] + df['bbox_y2']) / 2

df['grid_col'] = (df['center_x'] / IMG_WIDTH * GRID_COLS).astype(int)
df['grid_row'] = (df['center_y'] / IMG_HEIGHT * GRID_ROWS).astype(int)

df['grid_col'] = df['grid_col'].clip(0, GRID_COLS - 1)
df['grid_row'] = df['grid_row'].clip(0, GRID_ROWS - 1)

output_csv_path = "/content/data/moto_grid_mapeamento.csv"
df[['arquivo', 'moto_id', 'grid_row', 'grid_col']].to_csv(output_csv_path, index=False)


